In [ ]:
import os

from astropy.io import fits
import astropy.table as at
import astropy.coordinates as coord
import astropy.units as u

import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from tqdm.notebook import tqdm
import thejoker as tj
import lightkurve as lk

import pymc3 as pm
import theano.tensor as tt
import exoplanet as xo

from hq.data import get_rvdata
from hq.samples_analysis import extract_MAP_sample

In [ ]:
kepid = 'KIC 2010607'
apid = '2M19220135+3727324'

In [ ]:
allvisit = fits.getdata('/mnt/home/apricewhelan/data/APOGEE_DR16/allVisit-r12-l33.fits')
gold = at.QTable(at.Table.read('../catalogs/gold_sample.fits').filled())
row = gold[gold['APOGEE_ID'] == apid]

In [ ]:
visits = allvisit[allvisit['APOGEE_ID'] == apid]
rv_data = get_rvdata(visits)

In [ ]:
sample = extract_MAP_sample(row)
_ = tj.plot_phase_fold(data=rv_data, sample=sample)

In [ ]:
lcfs = lk.search_lightcurvefile(kepid, mission='Kepler').download_all()
stitched_lc = lcfs.PDCSAP_FLUX.stitch()

In [ ]:
# From DFM!
with lcfs[0].hdu as hdu:
    tpf_hdr = hdu[1].header
texp = tpf_hdr["FRAMETIM"] * tpf_hdr["NUM_FRM"]
texp /= 60.0 * 60.0 * 24.0  # days

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(stitched_lc.astropy_time.tcb.jd,
        stitched_lc.flux,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)

MAD = np.nanmedian(np.abs(stitched_lc.flux - np.nanmedian(stitched_lc.flux)))
std = 1.5 * MAD
mask = (stitched_lc.flux - 1) < 6*std
ax.plot(stitched_lc.astropy_time.tcb.jd[~mask],
        stitched_lc.flux[~mask],
        marker='x', mew=2, ls='none', ms=8, color='r', zorder=10)

lc = stitched_lc[mask]

In [ ]:
# Convert to parts per thousand
x = lc.astropy_time.tcb.jd
y = lc.flux
mu = np.median(y)
y = (y / mu - 1) * 1e3
yerr = lc.flux_err * 1e3

x_ref = np.min(x)
x = x - x_ref

In [ ]:
len(x)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(x, y,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)
ax.set_xlim(0, 500)

Radial velocity data, relative to the same reference time:

In [ ]:
x_rv = rv_data.t.tcb.jd - x_ref
y_rv = (rv_data.rv - row['MAP_v0'][0]).to_value(u.m/u.s)
yerr_rv = rv_data.rv_err.to_value(u.m/u.s)
mean_rv = np.mean(y_rv)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))
phase = (x / row['MAP_P'].value * 1.00025006 + 0.5) % 1. - 0.5
cc = ax.scatter(phase, y, c=lc.time,
                s=1, alpha=0.4, lw=1)
ax.set_xlabel('Phase')
ax.set_ylabel('Normalized Flux')

ax.set_xlim(-0.05, 0.25)
# ax.set_xlim(-0.5, 0.5)

cb = fig.colorbar(cc)
cb.set_label('Kepler time', fontsize=14)
fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
PP = row['MAP_P'].value / 1.00025006

fig, axes = plt.subplots(2, 1, figsize=(10, 8),
                         sharex=True)

ax = axes[0]
phase = (x / PP + 0.5) % 1. - 0.5
ax.plot(phase, y, 
        ms=1.5, ls='none', color='k', alpha=0.25)
# ax.set_xlabel('Phase')
ax.set_ylabel('Normalized Flux')

# ax.set_xlim(0.45, 0.75)
ax.set_xlim(-0.5, 0.5)

ax = axes[1]
phase = (x_rv / PP + 0.5) % 1. - 0.5
ax.errorbar(phase, y_rv/1e3, 
            yerr=np.sqrt((yerr_rv/1e3)**2 + row['MAP_s'].to_value(u.km/u.s)[0]**2), 
            marker='o', ls='none')

ax.set_xlabel('Phase')
ax.set_ylabel('RV [km/s]')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
from astropy.timeseries import BoxLeastSquares

m = np.zeros(len(x), dtype=bool)
period_grid = np.exp(np.linspace(np.log(sample['P'][0].value)-0.5, 
                                 np.log(sample['P'][0].value)+0.5, 
                                 10000))

bls = BoxLeastSquares(x[~m], y[~m])
bls_power = bls.power(period_grid, 0.1, oversample=20)

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 6))
ax.plot(x, y,
        marker='o', ls='none', mew=0, ms=1.5, alpha=0.5)
ax.set_xlim(0, 200)
for i in range(10):
    ax.axvline(bls_t0 + i*bls_period, color='tab:red', alpha=0.25)

Load primary stellar radius and mass from STARHORSE and Gaia:

In [ ]:
M_star = row['mass'][0], row['mass_err'][0]
R_star = row['radius_val'][0], row['radius_val'][0] - row['radius_percentile_lower'][0]
M_star, R_star

In [ ]:
msini = row['m2_min_50'][0].to(u.M_sun)
msini, msini.to(u.Mjup)

---

## Try fitting RV's only:

In [ ]:
rv_span = np.max(x_rv) - np.min(x_rv)
t_rv = np.arange(x_rv.min() - rv_span/10, 
                 x_rv.max() + rv_span/10, 
                 row['MAP_P'][0].to_value(u.day) / 128)


def build_rv_model(rv_t0, rv_period, rv_omega):
        
    with pm.Model() as model:

        BoundedNormal = pm.Bound(pm.Normal, lower=0.5, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])
        
        # Parameters of the companion
        logm = pm.Normal("logm", mu=np.log(msini.value), sd=1)  # companion mass
        m_pl = pm.Deterministic("m_pl", tt.exp(logm))
        
        # Parameters of the orbit
        logP = pm.Normal("logP", mu=np.log(rv_period), sd=1)
        t_peri = pm.Normal("t_peri", mu=rv_t0, sd=1, testval=rv_t0)
        ecc = xo.distributions.eccentricity.kipping13(
            "ecc", long=False, testval=0.59
        )
        omega = xo.distributions.Angle("omega", testval=rv_omega)
        period = pm.Deterministic("period", tt.exp(logP))  # for tracking

        # RV jitter
        logs_rv = pm.Normal("logs_rv", mu=np.log(500.), sd=0.5)  # MAP_s~350 m/s, but prob. bigger
        rv0 = pm.Normal("rv0", mu=0, sd=500.)  # m/s

        # Orbit model
        cosi = pm.Uniform('cosi', 0, 1)
        incl = pm.Deterministic('incl', tt.arccos(cosi))
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t_periastron=t_peri,
            incl=incl,
            m_planet=xo.units.with_unit(m_pl, msini.unit),
            ecc=ecc,
            omega=omega,
        )
        
        # Evaluate orbit model
        vrad = orbit.get_radial_velocity(x_rv)

        # Evaluate the likelihood for the RVs
        rv_model = pm.Deterministic("rv_model", vrad + rv0)
        err = tt.sqrt(yerr_rv ** 2 + tt.exp(2 * logs_rv))
        pm.Normal("rv_obs", mu=rv_model, sd=err, observed=y_rv)

        # Evaluate the RV model on a finer grid:
        pm.Deterministic("rv_model_pred", 
                         orbit.get_radial_velocity(t_rv) + rv0)
        
#         map_soln = model.test_point
#         map_soln = xo.optimize(start=map_soln, vars=[logP, t0])
#         map_soln = xo.optimize(start=map_soln, vars=[logP, t0, omega, ecc, rv0, logs_rv])
#         map_soln = xo.optimize(start=map_soln)

#     return model, map_soln
    return model, None

In [ ]:
rv_M0 = row['MAP_M0'][0]
rv_P = row['MAP_P'][0].to_value(u.day)
rv_omega = row['MAP_omega'][0].to(u.rad)

rv_t0 = rv_data.t0 - Time(x_ref, format='jd', scale='tcb')
rv_t0 = rv_t0.jd - rv_M0 / (2*np.pi*u.rad) * rv_P
rv_t0 = rv_t0.value
rv_t0

In [ ]:
for i in [0]:
    _rv_model, _rv_soln = build_rv_model(rv_t0 + i, 
                                         rv_period=rv_P,
                                         rv_omega=rv_omega)

    with _rv_model:
        rv_pred = xo.eval_in_model(_rv_model.rv_model_pred)

    plt.plot(t_rv % rv_P, rv_pred)
plt.scatter(x_rv % rv_P, y_rv)

In [ ]:
rv_P, np.log(rv_P)

In [ ]:
def plot_rv_curve(soln):
    err = np.sqrt(yerr_rv ** 2 + np.exp(2 * soln["logs_rv"]))
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 7))

    ax = axes[0]
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.plot((t_rv/soln['period']) % 1, soln["rv_model_pred"], label="model")
    ax.legend(fontsize=10)
    ax.set_ylabel("radial velocity [m/s]", fontsize=y_fontsize)
    ax.set_xlim(0, 1)

    ax = axes[1]
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.axhline(0, color="k", lw=1)
    ax.set_ylabel("residuals [m/s]", fontsize=y_fontsize)
    ax.set_xlim(t_rv.min(), t_rv.max())
    ax.set_xlabel("time [days]")
    fig.tight_layout()
    
_rv_soln["logs_rv"] = np.log(500.)
plot_rv_curve(_rv_soln)

---

In [ ]:
Rs = R_star[0] * u.Rsun
a = 0.14*u.AU
TD = 0.18 * u.day
P = row['MAP_P'][0]
b = 1.
with u.set_enabled_equivalencies(u.dimensionless_angles()):
    Rp = np.sqrt((a * np.sin(np.pi*TD/P))**2 + (b*Rs)**2) - Rs
Rp.to(u.Rsun)

In [ ]:
elem = TwoBodyKeplerElements(P=row['MAP_P'][0], e=0.6, 
                             m1=M_star[0]*u.Msun,
                             m2=0.25*u.Msun, 
                             omega=0*u.rad, i=90*u.deg)
orbit1 = KeplerOrbit(elem.primary)
orbit2 = KeplerOrbit(elem.secondary)
ts = rv_data.t0 + np.linspace(0, 30, 8192)*u.day

RR = orbit1.reference_plane(ts).data.without_differentials() - orbit2.reference_plane(ts).data.without_differentials()
plt.plot((RR.norm() / (R_star[0]*u.Rsun)).decompose())

---

In The Joker, t0 is pericenter of the primary, but in exoplanet it is the secondary? For some reason, our definitions of t0 are off by 1/2 period...

In [ ]:
rv_M0 = row['MAP_M0'][0]
rv_P = row['MAP_P'][0].to_value(u.day)
rv_omega = row['MAP_omega'][0].to(u.rad)

rv_t0 = rv_data.t0 - Time(x_ref, format='jd', scale='tcb')
rv_t0 = rv_t0.jd - rv_M0 / (2*np.pi*u.rad) * rv_P
rv_t0 = rv_t0.value
rv_t0

## Try fitting light curve only:

In [ ]:
def build_lc_model(phot_mask=None, start=None):
    if phot_mask is None:
        phot_mask = np.ones(len(x), dtype=bool)
        
    with pm.Model() as model:

        # Parameters for the stellar properties of the primary
        BoundedNormal = pm.Bound(pm.Normal, lower=0.5, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])

        # Parameters of the companion
        logm = pm.Normal("logm", mu=np.log(msini.value), sd=1)  # companion mass
        m_pl = pm.Deterministic("m_pl", tt.exp(logm))
        
        # Parameters of the orbit
        logP = pm.Normal("logP", mu=np.log(rv_P), sd=1)
        t_peri = pm.Normal("t_peri", mu=rv_t0, sd=1, testval=rv_t0)
        
        ecc = pm.Bound(pm.Normal, lower=0, upper=1)("ecc", mu=0.59, sd=0.05, testval=0.59)
#         ecc = xo.distributions.eccentricity.kipping13(
#             "ecc", long=False, testval=0.59
#         )
        omega = xo.distributions.Angle("omega", testval=rv_omega)
        period = pm.Deterministic("period", tt.exp(logP))  # for tracking

        # Flux jitter & GP parameters
        logs2 = pm.Normal("logs2", mu=np.log(np.var(y[phot_mask])), sd=2)
        logw0 = pm.Normal("logw0", mu=0., sd=3)
        # logSw4 = pm.Normal("logSw4", mu=np.log(np.var(y[phot_mask])), sd=0.2)
        # logSw4 = pm.Normal("logSw4", mu=-4.5, sd=1)
        logSw4 = pm.Constant("logSw4", -10., testval=-10)

        # Orbit model
        cosi = pm.Uniform('cosi', 0, 1)
        incl = pm.Deterministic('incl', tt.arccos(cosi))
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t_periastron=t_peri,
            incl=incl,
            m_planet=xo.units.with_unit(m_pl, msini.unit),
            ecc=ecc,
            omega=omega,
        )
        
        # ==============================================================
        # Light curve:
        #
        mean_flux = pm.Normal("mean_flux", mu=0.0, sd=1.0)
        
        # Compute the Heartbeat model
        sinf, cosf = orbit._get_true_anomaly(x[phot_mask])
        f = tt.arctan2(sinf, cosf)[:, 0]
        pm.Deterministic("f", f)
        dX, dY, dZ = orbit.get_relative_position(x[phot_mask])
        R = tt.sqrt(dX**2 + dY**2 + dZ**2)
        logS = pm.Normal('logS', mu=-2.5, sd=0.1, testval=-2.5)
        S = pm.Deterministic('S', tt.exp(logS))
        hb_light_curve = S * (
            (1 - 3 * tt.sin(incl)**2 * tt.sin(f - omega)**2) / (R / orbit.a)**3
        )
        pm.Deterministic("heartbeat_light_curve", hb_light_curve)
        
        # GP model for the light curve
        # Set up the Gaussian Process model
        kernel = xo.gp.terms.SHOTerm(log_Sw4=logSw4, 
                                     log_w0=logw0, 
                                     Q=1 / np.sqrt(2))
        gp = xo.gp.GP(kernel, x[phot_mask], 
                      yerr[phot_mask]**2 + tt.exp(logs2))
        
        pm.Potential("transit_obs", 
                     gp.log_likelihood(y[phot_mask] - (hb_light_curve + mean_flux)))
        pm.Deterministic("gp_pred", gp.predict())

        # Fit for the maximum a posteriori parameters, I've found that I can get
        # a better solution by trying different combinations of parameters in turn
        if start is None:
            start = model.test_point
        map_soln = xo.optimize(start=start, vars=[logP, t_peri])
        map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri, omega, ecc])
        map_soln = xo.optimize(start=map_soln, vars=[logS])
        map_soln = xo.optimize(start=map_soln, vars=[logs2, logSw4, logw0])
        map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri, omega, ecc])
        map_soln = xo.optimize(start=map_soln)

    return model, map_soln

In [ ]:
sub_mask = np.zeros(len(x), dtype=bool)
sub_mask[:15000] = True
sub_mask &= np.isfinite(y)

lc_model, lc_map_soln = build_lc_model(phot_mask=sub_mask)

In [ ]:
y_fontsize = 12

def plot_light_curve(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, axes = plt.subplots(4, 1, figsize=(12, 12), sharex=True, sharey=True)

    ax = axes[0]
    ax.plot(x[mask], y[mask], "k", label="data")
    gp_mod = soln["gp_pred"] + soln["mean_flux"]
    ax.plot(x[mask], gp_mod, color="C2", label="gp model")
    ax.legend(fontsize=10)
    ax.set_ylabel("relative flux [ppt]", fontsize=y_fontsize)

    ax = axes[1]
    ax.plot(x[mask], y[mask] - gp_mod, "k")
    ax.set_ylabel("de-trended flux [ppt]", fontsize=y_fontsize)
    
    ax = axes[2]
    ax.plot(x[mask], soln["heartbeat_light_curve"], "k")
    ax.set_ylabel("heartbeat [ppt]", fontsize=y_fontsize)

    ax = axes[3]
    mod = gp_mod + soln["heartbeat_light_curve"]
    ax.plot(x[mask], y[mask] - mod, "k")
    ax.axhline(0, color="#aaaaaa", lw=1)
    ax.set_ylabel("residuals [ppt]", fontsize=y_fontsize)
    ax.set_xlim(x[mask].min(), x[mask].max())
    ax.set_xlabel("time [days]")
    
    fig.set_facecolor('w')
    fig.tight_layout()

    return fig

In [ ]:
lc_map_soln

In [ ]:
_ = plot_light_curve(lc_map_soln, sub_mask)

## Try fitting light curve without GP:

In [ ]:
def build_hb_only_lc_model(phot_mask=None, start=None):
    if phot_mask is None:
        phot_mask = np.ones(len(x), dtype=bool)
        
    with pm.Model() as model:

        # Parameters for the stellar properties of the primary
        BoundedNormal = pm.Bound(pm.Normal, lower=0.5, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])

        # Parameters of the companion
        logm = pm.Normal("logm", mu=np.log(msini.value), sd=1)  # companion mass
        m_pl = pm.Deterministic("m_pl", tt.exp(logm))
        
        # Parameters of the orbit
        logP = pm.Normal("logP", mu=np.log(rv_P), sd=1)
        t_peri = pm.Normal("t_peri", mu=rv_t0, sd=1, testval=rv_t0)
        
        ecc = pm.Bound(pm.Normal, lower=0, upper=1)(
            "ecc", mu=0.59, sd=0.05, testval=0.59)
        omega = xo.distributions.Angle("omega", testval=rv_omega)
        period = pm.Deterministic("period", tt.exp(logP))  # for tracking

        # Orbit model
        cosi = pm.Uniform('cosi', 0, 1)
        incl = pm.Deterministic('incl', tt.arccos(cosi))
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t_periastron=t_peri,
            incl=incl,
            m_planet=xo.units.with_unit(m_pl, msini.unit),
            ecc=ecc,
            omega=omega,
        )
        
        # ==============================================================
        # Light curve:
        #
        mean_flux = pm.Normal("mean_flux", mu=0.0, sd=1.0)
        
        # Compute the Heartbeat model
        sinf, cosf = orbit._get_true_anomaly(x[phot_mask])
        f = tt.arctan2(sinf, cosf)[:, 0]
        pm.Deterministic("f", f)
        dX, dY, dZ = orbit.get_relative_position(x[phot_mask])
        R = tt.sqrt(dX**2 + dY**2 + dZ**2)
        logS = pm.Normal('logS', mu=-2.5, sd=0.1, testval=-2.5)
        S = pm.Deterministic('S', tt.exp(logS))
        hb_light_curve = S * (
            (1 - 3 * tt.sin(incl)**2 * tt.sin(f - omega)**2) / (R / orbit.a)**3
        )
        pm.Deterministic("heartbeat_light_curve", hb_light_curve)
        
        pm.Normal("obs", 
                  mu=hb_light_curve + mean_flux, 
                  sd=yerr[phot_mask], 
                  observed=y[phot_mask])

        # Fit for the maximum a posteriori parameters, I've found that I can get
        # a better solution by trying different combinations of parameters in turn
        map_soln = model.test_point
#         map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri])
#         map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri, omega, ecc])
#         map_soln = xo.optimize(start=map_soln, vars=[logS])
#         map_soln = xo.optimize(start=map_soln)

    return model, map_soln

In [ ]:
sub_mask = np.zeros(len(x), dtype=bool)
sub_mask[:15000] = True
sub_mask &= np.isfinite(y)

lc_model, lc_map_soln = build_hb_only_lc_model(phot_mask=sub_mask)

In [ ]:
with lc_model:
    derp = xo.eval_in_model(lc_model.heartbeat_light_curve)
    
derp.shape

In [ ]:
map_estimate = pm.find_MAP(model=lc_model, method='powell')

In [ ]:
map_estimate

In [ ]:
y_fontsize = 12

def plot_hb_only_light_curve(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, axes = plt.subplots(3, 1, figsize=(12, 12), 
                             sharex=True, sharey=True)

    ax = axes[0]
    ax.plot(x[mask], y[mask], "k", label="data")
    
    ax = axes[1]
    ax.plot(x[mask], soln["heartbeat_light_curve"], "k")
    ax.set_ylabel("heartbeat [ppt]", fontsize=y_fontsize)

    ax = axes[2]
    mod = soln['mean_flux'] + soln["heartbeat_light_curve"]
    ax.plot(x[mask], y[mask] - mod, "k")
    ax.axhline(0, color="#aaaaaa", lw=1)
    ax.set_ylabel("residuals [ppt]", fontsize=y_fontsize)
    ax.set_xlim(x[mask].min(), x[mask].max())
    ax.set_xlabel("time [days]")
    
    fig.set_facecolor('w')
    fig.tight_layout()

    return fig

In [ ]:
# _ = plot_hb_only_light_curve(lc_map_soln, sub_mask)
_ = plot_hb_only_light_curve(map_estimate, sub_mask)

---

## Fit light curve and RVs

In [ ]:
rv_span = np.max(x_rv) - np.min(x_rv)
t_rv = np.arange(x_rv.min() - rv_span/10, 
                 x_rv.max() + rv_span/10, 
                 row['MAP_P'][0].to_value(u.day) / 128)


def build_model(phot_mask=None, start=None):
    if phot_mask is None:
        phot_mask = np.ones(len(x), dtype=bool)
        
    with pm.Model() as model:

        # Parameters for the stellar properties of the primary
        BoundedNormal = pm.Bound(pm.Normal, lower=0.5, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])

        # Parameters of the companion
        logm = pm.Normal("logm", mu=np.log(msini.value), sd=1)  # companion mass
        m_pl = pm.Deterministic("m_pl", tt.exp(logm))
        
        # Parameters of the orbit
        logP = pm.Normal("logP", mu=np.log(rv_P), sd=1)
        t_peri = pm.Normal("t_peri", mu=rv_t0, sd=1, testval=rv_t0)
        ecc = xo.distributions.eccentricity.kipping13(
            "ecc", long=False, testval=0.59
        )
        omega = xo.distributions.Angle("omega", testval=rv_omega)
        period = pm.Deterministic("period", tt.exp(logP))  # for tracking

        # RV jitter
        logs_rv = pm.Normal("logs_rv", mu=np.log(500.), sd=0.1)  # MAP_s~350 m/s, but prob. bigger
        rv0 = pm.Normal("rv0", mu=0, sd=500.)  # m/s

        # Flux jitter & GP parameters
        logs2 = pm.Normal("logs2", mu=np.log(np.var(y[phot_mask])), sd=2)
        logw0 = pm.Normal("logw0", mu=0., sd=3)
        # logSw4 = pm.Normal("logSw4", mu=np.log(np.var(y[phot_mask])), sd=0.2)
        logSw4 = pm.Normal("logSw4", mu=-4.5, sd=1)

        # Orbit model
        cosi = pm.Uniform('cosi', 0, 1)
        incl = pm.Deterministic('incl', tt.arccos(cosi))
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t_periastron=t_peri,
            incl=incl,
            m_planet=xo.units.with_unit(m_pl, msini.unit),
            ecc=ecc,
            omega=omega,
        )
        
        # ==============================================================
        # Light curve:
        #
        mean_flux = pm.Normal("mean_flux", mu=0.0, sd=1.0)
        
        # Compute the Heartbeat model
        sinf, cosf = orbit._get_true_anomaly(x[phot_mask])
        f = tt.arctan2(sinf, cosf)[:, 0]
        pm.Deterministic("f", f)
        dX, dY, dZ = orbit.get_relative_position(x[phot_mask])
        R = tt.sqrt(dX**2 + dY**2 + dZ**2)
        logS = pm.Normal('logS', mu=-2.5, sd=0.1, testval=-2.5)
        S = pm.Deterministic('S', tt.exp(logS))
        hb_light_curve = S * (
            (1 - 3 * tt.sin(incl)**2 * tt.sin(f - omega)**2) / (R / orbit.a)**3
        )
        pm.Deterministic("heartbeat_light_curve", hb_light_curve)
        
        # GP model for the light curve
        # Set up the Gaussian Process model
        kernel = xo.gp.terms.SHOTerm(log_Sw4=logSw4, 
                                     log_w0=logw0, 
                                     Q=1 / np.sqrt(2))
        gp = xo.gp.GP(kernel, x[phot_mask], 
                      yerr[phot_mask]**2 + tt.exp(logs2))
        
        pm.Potential("transit_obs", 
                     gp.log_likelihood(y[phot_mask] - (hb_light_curve + mean_flux)))
        pm.Deterministic("gp_pred", gp.predict())
        
        # ==============================================================
        # Radial velocity:
        #
        # Set up the RV model and save it as a deterministic
        # for plotting purposes later
        vrad = orbit.get_radial_velocity(x_rv)
        pm.Deterministic("vrad", vrad)

        # The likelihood for the RVs
        rv_model = pm.Deterministic("rv_model", vrad + rv0)
        err = tt.sqrt(yerr_rv ** 2 + tt.exp(2 * logs_rv))
        pm.Normal("obs", mu=rv_model, sd=err, observed=y_rv)

        vrad_pred = orbit.get_radial_velocity(t_rv)
        pm.Deterministic("vrad_pred", vrad_pred)
        bkg_pred = pm.Deterministic("bkg_pred", rv0)
        pm.Deterministic("rv_model_pred", vrad_pred + bkg_pred)

        # Fit for the maximum a posteriori parameters, I've found that I can get
        # a better solution by trying different combinations of parameters in turn
        if start is None:
            start = model.test_point
        map_soln = xo.optimize(start=start, vars=[logP, t_peri])
        map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri, omega, ecc, rv0, logs_rv])
        map_soln = xo.optimize(start=map_soln, vars=[logS])
        map_soln = xo.optimize(start=map_soln, vars=[logs2, logSw4])
        map_soln = xo.optimize(start=map_soln, vars=[logw0])
        map_soln = xo.optimize(start=map_soln, vars=[logP, t_peri, omega, ecc, rv0, logs_rv])
        map_soln = xo.optimize(start=map_soln)

    return model, map_soln

In [ ]:
sub_mask = np.zeros(len(x), dtype=bool)
sub_mask[:15000] = True
sub_mask &= np.isfinite(y)

In [ ]:
model0, map_soln0 = build_model(phot_mask=sub_mask)

In [ ]:
# np.exp(map_soln0['logs_rv'])
map_soln0['ecc'], map_soln0['logS']

In [ ]:
np.exp(map_soln0['logSw4']), map_soln0['logSw4']

In [ ]:
y_fontsize = 12

def plot_light_curve(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, axes = plt.subplots(4, 1, figsize=(12, 12), sharex=True, sharey=True)

    ax = axes[0]
    ax.plot(x[mask], y[mask], "k", label="data")
    gp_mod = soln["gp_pred"] + soln["mean_flux"]
    ax.plot(x[mask], gp_mod, color="C2", label="gp model")
    ax.legend(fontsize=10)
    ax.set_ylabel("relative flux [ppt]", fontsize=y_fontsize)

    ax = axes[1]
    ax.plot(x[mask], y[mask] - gp_mod, "k")
    ax.set_ylabel("de-trended flux [ppt]", fontsize=y_fontsize)
    
    ax = axes[2]
    ax.plot(x[mask], soln["heartbeat_light_curve"], "k")
    ax.set_ylabel("heartbeat [ppt]", fontsize=y_fontsize)

    ax = axes[3]
    mod = gp_mod + soln["heartbeat_light_curve"]
    ax.plot(x[mask], y[mask] - mod, "k")
    ax.axhline(0, color="#aaaaaa", lw=1)
    ax.set_ylabel("residuals [ppt]", fontsize=y_fontsize)
    ax.set_xlim(x[mask].min(), x[mask].max())
    ax.set_xlabel("time [days]")
    
    fig.tight_layout()

    return fig

def plot_folded_residual(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    mod = soln["gp_pred"] + soln["mean_flux"] + soln["heartbeat_light_curve"]
    resid = y[mask] - mod
    phase = x[mask] % soln['period']
    ax.plot(phase, y[mask] - mod, 
            marker='o', mew=0, ms=2., alpha=0.4,
            ls='none', color="k", label="residual")
    ax.legend(fontsize=10)
    ax.set_ylabel("relative flux [ppt]", fontsize=y_fontsize)

    ax.set_xlabel("time [days]")
    
    fig.tight_layout()

    return fig

def plot_rv_curve(soln):
    err = np.sqrt(yerr_rv ** 2 + np.exp(2 * soln["logs_rv"]))
    
    fig, axes = plt.subplots(2, 1, figsize=(10, 7))

    ax = axes[0]
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar((x_rv/soln['period']) % 1, y_rv, yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.plot((t_rv/soln['period']) % 1, soln["vrad_pred"], "--k", alpha=0.5, color='tab:blue')
    ax.plot((t_rv/soln['period']) % 1, soln["rv_model_pred"], label="model")
    ax.legend(fontsize=10)
    ax.set_ylabel("radial velocity [m/s]", fontsize=y_fontsize)
    ax.set_xlim(0, 1)

    ax = axes[1]
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=yerr_rv, 
                marker='o', ls='none', color='k')
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=err, 
                marker='', ls='none', color='tab:red', zorder=-100)
    ax.axhline(0, color="k", lw=1)
    ax.set_ylabel("residuals [m/s]", fontsize=y_fontsize)
    ax.set_xlim(t_rv.min(), t_rv.max())
    ax.set_xlabel("time [days]")
    fig.tight_layout()

In [ ]:
fig = plot_light_curve(map_soln0, mask=sub_mask)
fig.axes[0].set_xlim(0, 300)
fig.axes[0].set_ylim(-1, 1)
fig.set_facecolor('w')

In [ ]:
plot_rv_curve(map_soln0)

Sigma clip the outliers:

In [ ]:
_mask = sub_mask

mod = (
    map_soln0["gp_pred"]
    + map_soln0["mean_flux"]
    + map_soln0["heartbeat_light_curve"]
)
resid = y[_mask] - mod
rms = np.sqrt(np.median(resid ** 2))
mask_iter1 = np.abs(resid) < 8 * rms

plt.plot(x[_mask], resid, "k", label="data")
plt.plot(x[_mask][~mask_iter1], resid[~mask_iter1], "xr", mew=1, label="outliers")
plt.axhline(0, color="#aaaaaa", lw=1)
plt.ylabel("residuals [ppt]")
plt.xlabel("time [days]")
plt.legend(fontsize=12, loc=4)
plt.xlim(x[_mask].min(), x[_mask].max());
plt.axhline(7*rms)
plt.axhline(-7*rms)

In [ ]:
tmp_mask = sub_mask.copy()
tmp_mask[:len(mask_iter1)] = mask_iter1
model, map_soln1 = build_model(start=map_soln0, phot_mask=tmp_mask)

In [ ]:
fig = plot_light_curve(map_soln1, tmp_mask);
fig.axes[0].set_xlim(0, 100)
fig.axes[0].set_ylim(-1, 1)
fig.set_facecolor('w')

In [ ]:
def plot_folded_residual(soln, mask=None):
    if mask is None:
        mask = np.ones(len(x), dtype=bool)

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))

    mod = soln["gp_pred"] + soln["mean_flux"] + soln["heartbeat_light_curve"]
    resid = y[mask] - mod
    phase = ((x[mask] - soln['t_peri']) / soln['period'] + 0.5) % 1. - 0.5
    ax.plot(phase, y[mask] - mod, 
            marker='o', mew=0, ms=2., alpha=1,
            ls='none', color="k", label="residual")
    ax.legend(fontsize=10)
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylabel("residual flux [ppt]")
    ax.set_xlabel("phase")
    
    fig.tight_layout()

    return fig

fig = plot_folded_residual(map_soln1, tmp_mask)
fig.axes[0].set_xlim(-0.1, 0.1)
fig.set_facecolor('w')

In [ ]:
plot_rv_curve(map_soln1)
plt.gcf().set_facecolor('w')

In [ ]:
np.degrees(map_soln1['incl'] * u.rad)

In [ ]:
row['m2_min_50']

In [ ]:
map_soln1['m_pl']

In [ ]:
row['LOGG']

In [ ]:
dflux = 0.15 / 1e3
(np.sqrt(dflux) * R_star[0] * u.Rsun).to(u.Rjup)

In [ ]:
np.exp(map_soln1['logP'])

In [ ]:
map_soln1['ecc']

In [ ]:
map_soln1['r_pl']

In [ ]:
mass = map_soln1['m_pl'] * u.Msun
mass.to(u.Mjup)

In [ ]:
map_soln1['rotperiod']

---

In [ ]:
np.random.seed(123)
with model:
    trace = pm.sample(
        tune=1000,
        draws=1000,
        start=map_soln1,
        chains=4,
        step=xo.get_dense_nuts_step(target_accept=0.9),
    )

In [ ]:
pm.summary(trace, var_names=["period", "r_pl", "m_pl", "ecc", "omega", "b", "rotperiod"])

In [ ]:
print(trace.varnames)

In [ ]:
np.mean((trace['r_pl'] * u.Rsun).to(u.Rjup))

In [ ]:
(np.mean(trace['m_pl'])*u.Msun).to(u.Mjup), (np.std(trace['m_pl'])*u.Msun).to(u.Mjup)

In [ ]:
np.mean((trace['rotperiod'] * u.day))

In [ ]:
np.mean(trace['ecc'])

In [ ]:
np.mean(trace['m_star'])

In [ ]:
plt.figure(figsize=(8, 5))

# Get the posterior median orbital parameters
p = np.median(trace["period"])
t0 = np.median(trace["t0"])

# Plot the folded data
x_fold = (x_rv - t0 + 0.5 * p) % p - 0.5 * p
plt.errorbar(x_fold, y_rv, yerr=yerr_rv, fmt=".k", label="data")

# Compute the posterior prediction for the folded RV model for this
# planet
t_fold = (t_rv - t0 + 0.5 * p) % p - 0.5 * p
inds = np.argsort(t_fold)
pred = np.percentile(trace["vrad_pred"][:, inds], [16, 50, 84], axis=0)
plt.plot(t_fold[inds], pred[1], color="C1", label="model", marker='')
art = plt.fill_between(t_fold[inds], pred[0], pred[2], color="C1", alpha=0.3)
art.set_edgecolor("none")

plt.legend(fontsize=10)
plt.xlim(-0.5 * p, 0.5 * p)
plt.xlabel("phase [days]")
plt.ylabel("radial velocity [m/s]")
# plt.title("K2-24{0}".format(letter));

In [ ]:
plt.figure(figsize=(8, 5))

# Compute the GP prediction
gp_mod = np.median(trace["gp_pred"] + trace["mean_flux"][:, None], axis=0)

# Get the posterior median orbital parameters
p = np.median(trace["period"])
t0 = np.median(trace["t0"])

# Plot the folded data
x_fold = (x[tmp_mask] - t0 + 0.5 * p) % p - 0.5 * p
plt.plot(x_fold, y[tmp_mask] - gp_mod, ".k", label="data", zorder=-1000)

# Plot the folded model
inds = np.argsort(x_fold)
inds = inds[np.abs(x_fold)[inds] < 0.3]
pred = trace["transit_light_curves"][:, inds, 0]
pred = np.percentile(pred, [16, 50, 84], axis=0)
plt.plot(x_fold[inds], pred[1], color="C1", label="model", marker='')
art = plt.fill_between(
    x_fold[inds], pred[0], pred[2], color="C1", alpha=0.5, zorder=1000
)
art.set_edgecolor("none")

# Annotate the plot with the planet's period
txt = "period = {0:.4f} +/- {1:.4f} d".format(
    np.mean(trace["period"]), np.std(trace["period"])
)

plt.xlim(-0.5 * p, 0.5 * p)
plt.xlabel("time since transit [days]")
plt.ylabel("de-trended flux")
# plt.title("K2-24{0}".format(letter))
plt.xlim(-0.3, 0.3)